In [1]:
try:
    sc.install_pypi_package("boto3")
except:
    print('Packages already installed')

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1638640742135_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


You are using pip version 9.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [2]:
sc.install_pypi_package("cython")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


You are using pip version 9.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [3]:
sc.install_pypi_package("pandas==0.25.1") #Install pandas version 0.25.1 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


You are using pip version 9.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [4]:
import boto3
import time
import pandas as pd
import os
import sys
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import col, regexp_replace
import re


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

aws-cfn-bootstrap (2.0)
beautifulsoup4 (4.9.3)
boto (2.49.0)
boto3 (1.20.20)
botocore (1.23.20)
click (7.1.2)
Cython (0.29.24)
docutils (0.14)
jmespath (0.10.0)
joblib (1.0.1)
lockfile (0.11.0)
lxml (4.6.2)
mysqlclient (1.4.2)
nltk (3.5)
nose (1.3.4)
numpy (1.16.5)
pandas (0.25.1)
pip (9.0.1)
py-dateutil (2.2)
pystache (0.5.4)
python-daemon (2.2.3)
python-dateutil (2.8.2)
python37-sagemaker-pyspark (1.4.1)
pytz (2021.1)
PyYAML (5.4.1)
regex (2021.3.17)
s3transfer (0.5.0)
setuptools (28.8.0)
simplejson (3.2.0)
six (1.13.0)
tqdm (4.59.0)
urllib3 (1.26.7)
wheel (0.29.0)
windmill (1.6)

DEPRECATION: The default format will switch to columns in the future. You can use --format=(legacy|columns) (or define a format=(legacy|columns) in your pip.conf under the [list] section) to disable this warning.
You are using pip version 9.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [11]:
s3 = boto3.resource('s3')
aws_bucket = s3.Bucket('datasets.elasticmapreduce')

for my_bucket_object in aws_bucket.objects.filter(Prefix='ngrams/books/20090715/eng-us-all/1gram/data'):
    print(my_bucket_object)
    words_obj = my_bucket_object

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3.ObjectSummary(bucket_name='datasets.elasticmapreduce', key='ngrams/books/20090715/eng-us-all/1gram/data')

In [12]:
class_bucket = s3.Bucket('political-movement-data')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
for my_bucket_object in class_bucket.objects.filter(Prefix='key_terms.csv'):
    terms_bucket = my_bucket_object

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
if sys.version_info[0] < 3: 
    from StringIO import StringIO # Python 2.x
else:
    from io import StringIO # Python 3.x

kt_string = terms_bucket.get()['Body'].read().decode('utf-8')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
terms_df = pd.read_csv(StringIO(kt_string))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
terms_df['key_terms'] = terms_df['Key Terms'].apply(lambda x: x.split(','))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
terms_df = terms_df.explode('key_terms')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
terms_df['key_terms'] = terms_df['key_terms'].str.strip()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
# Converting key terms to a list
term_list = terms_df['key_terms'].to_list()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
#Mapper to convert to row object
#Note: remove all characters besides letters and spaces and make lowercase
def mapper(line):
    fields= line.split("\t") 
    return Row(ngram=str(fields[0].lower()), 
               year=int(fields[1]), 
               occur=int(fields[2]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# Access data and convert into row object format
lines = sc.sequenceFile("s3://datasets.elasticmapreduce/ngrams/books/20090715/eng-us-all/1gram/data")
stringlines=lines.map(lambda x: x[1])
gramsdf = stringlines.map(mapper)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
gramsdf.take(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(ngram='#', occur=6, year=1584), Row(ngram='#', occur=1, year=1596)]

In [36]:
#Filter by year (2000+)
filtered=gramsdf.filter(lambda x: x['year']>=1945)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
filtered.take(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(ngram='#', occur=3635, year=1945), Row(ngram='#', occur=5684, year=1946)]

In [37]:
#Filter for only the terms in the terms_df dataframe
filtered_words=filtered.filter(lambda x: x['ngram'] in term_list)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
#Create data frame and table to query
df = spark.createDataFrame(filtered_words).cache()
df.createOrReplaceTempView("df")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
#Query for top terms in each year
test_words = sqlContext.sql('''SELECT ngram, year, sum(occur) as occur
                     FROM df
                     GROUP BY ngram, year
                     ORDER BY occur DESC''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
test_words.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----+------+
|       ngram|year| occur|
+------------+----+------+
|   communist|1967|150261|
|   communist|1968|124917|
|   communist|1965|121710|
|   communist|1966|121444|
|   communist|1969|113081|
|   communist|1962|111124|
|   communist|1964|109911|
|   communist|1963|108945|
|     liberal|2007|106473|
|   communist|1972|101326|
|   communist|1970| 98758|
|conservative|2007| 98279|
|   communist|1961| 95469|
|     liberal|2008| 93300|
|   communist|1971| 91807|
|     liberal|2006| 91752|
|     liberal|1968| 91551|
|     liberal|2001| 87646|
| progressive|2007| 87012|
| progressive|2006| 86427|
+------------+----+------+
only showing top 20 rows

In [46]:
#Save file to single csv file in desired output bucket
test_words.coalesce(1).write.csv("s3://political-movement-data/output/political_ngrams.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
# Check the csv is in the bucket
for my_bucket_object in class_bucket.objects.filter(Prefix='output/political_ngrams.csv'):
    print(my_bucket_object)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3.ObjectSummary(bucket_name='political-movement-data', key='output/political_ngrams.csv/_SUCCESS')
s3.ObjectSummary(bucket_name='political-movement-data', key='output/political_ngrams.csv/part-00000-3f870e84-efd3-4055-8822-a947b2474014-c000.csv')